In [13]:
import os
import time
import requests
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import threading
import queue

# set the year we need & get all teams in NBA

In [14]:
#create a list from 2007-2018
year = []
for i in range(2007,2019):
    year.append(i)
#Get 30 teams in NBA
url = 'https://www.basketball-reference.com/'
table = pd.read_html(url)
for i in range(len(table)):
    if len(table[i]) >= 15:
        if i == 0:
            East = table[i]
        else:
            West = table[i]
East = East.East
East = list(East)
West = West.West
West = list(West)
nbateam = []
nbateam.append(East)
nbateam.append(West)
#filter other words in both east and west
for t in nbateam:
    for i in range(len(t)):
        count = 0
        team = ''
        for j in t[i]:
            count = count + 1
            team = team + j
            if count == 3:
                t[i] = team
                break
# for i in range(len(West)):
#     count = 0
#     team = ''
#     for j in West[i]:
#         count = count + 1
#         team = team + j
#         if count == 3:
#             West[i] = team
#             break

# get 2007~2018 nba players list in each team

In [4]:
tStart = time.time()
Players1 = pd.DataFrame(columns=['name', 'webname', 'team', 'year', 'side', 'website'])
path = 'C:\\Users\\ecom\\Desktop\\NBAData'
if not os.path.isdir(path):
    os.mkdir(path)

for i in year:
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i)
    if not os.path.isdir(path):
        os.mkdir(path)
        
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s
        if not os.path.isdir(path):
            os.mkdir(path)
        path1 = path

        for j in nbateam[t]:
            
            #change teamname
            teamname = j
            if i <= 2012 and j == 'BRK':
                j = 'NJN'
            elif i <= 2014 and j == 'CHO':
                j = 'CHA'
            elif i == 2007 and j == 'NOP':
                j = 'NOK'
            elif i >= 2008 and i <= 2013 and j == 'NOP':
                j = 'NOH'
            elif i <= 2008 and j == 'OKC':
                j = 'SEA'
                
            path = path1 + '\\' + teamname
            if not os.path.isdir(path):
                os.mkdir(path)
            webplayerlist = []
            url = 'https://www.basketball-reference.com/teams/' + j + '/' + str(i) + '.html'
                
            # print(url)
            table = pd.read_html(url)
            playerlist = list(table[0].Player)
            playerlist.sort()
            tm = []
            yr = []
            sd = []
            website = []

            pageSource = requests.get(url)
            sp = BeautifulSoup(pageSource.text, "lxml")
            table = sp.select('td.left')
            na = []
            np = []
            for f in range(len(table)):
                ul = table[f].get('data-append-csv')
                pn = table[f].text
                na.append(ul)
                np.append(pn)

            while None in na:
                na.remove(None)
            na1 = list(set(na))
            na1.sort(key=na.index)

            cal = 0
            np1 = []
            for v in np:
                if cal == 0:
                    np1.append(v)
                    cal = cal + 1
                elif cal != 0 and cal % 3 == 0:
                    np1.append(v)
                    cal = cal + 1
                else:
                    cal = cal + 1
                    continue
            
            for nu in range(len(na1)):
                tm.append(j)
                yr.append(str(i))
                sd.append(s)
            for wb in na1:
                for wba in wb:
                    letter = wba
                    break
                url1 = 'https://www.basketball-reference.com/players/' + letter + '/' + wb + '/gamelog/' + str(i) + '/'
                website.append(url1)
            
            Players = pd.DataFrame(columns=['name', 'webname', 'team', 'year', 'side', 'website'])
            Players.name = np1
            Players.webname = na1
            Players.team = tm
            Players.year = yr
            Players.side = sd
            Players.website = website
            Players1 = Players1.append(Players, ignore_index=True)
Players1.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 20 mins

Cost 1639.274997 s


# get each player's gamelog from season 2007 to 2018

In [ ]:
erroryear = []
errorteam = []
errorname = []
errortable = []
urlQueue = queue.Queue()
dfplayerlist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
web = list(dfplayerlist.website)
for url in web:
    urlQueue.put(url)
def getpersonaldata(n):
    while True:
        try:
            url = n.get_nowait()
            i = n.qsize()
        except Exception as e:
            break
        try:
            tables = pd.read_html(url)
            for j in range(len(dfplayerlist.website)):
                if dfplayerlist.website[j] == url:
                    pyear = dfplayerlist.year[j]
                    pside = dfplayerlist.side[j]
                    pteam = dfplayerlist.team[j]
                    pname = dfplayerlist.name[j]
            lntable = 0
            for ts in tables:
                if len(ts) > lntable:
                    lntable = len(ts)
                    table = ts
            gcol = []
            count = 0
            for c in table['G']:
                if c == 'G':
                    gcol.append(count)
                count = count + 1
            ctable = table.drop(gcol)
            ctable = ctable.reset_index()
            ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\' + str(pyear) + '\\' + pside + '\\' + pteam + '\\' + pname + '.xlsx')
        except Exception as e:
            erroryear.append(pyear)
            errorteam.append(pteam)
            errorname.append(pname)
            errortable.append(url)
            continue

if __name__=='__main__':
    time_start=time.time()
    thr = []
    thrnum = 10
    for i in range(thrnum):
        t = threading.Thread(target=getpersonaldata, args=(urlQueue,))
        thr.append(t)
    for t in thr:
        t.start()
    for t in thr:
        t.join()

if len(erroryear) != 0:
    errordic = {'year' : erroryear, 'team' : errorteam, 'name' : errorname, 'website' : errortable}
    error = pd.DataFrame(errordic)
    error.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error.xlsx')
time_end=time.time()
print('time cost',time_end-time_start,'s')

# Rebuild NBA data folder (If needed)

In [15]:
#rebuild folder
tStart = time.time()

path = 'C:\\Users\\ecom\\Desktop\\NBAData'
if not os.path.isdir(path):
    os.mkdir(path)

for i in year:
    path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i)
    if not os.path.isdir(path):
        os.mkdir(path)
        
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        path = 'C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s
        if not os.path.isdir(path):
            os.mkdir(path)
        path1 = path

        for j in nbateam[t]:
            
            #change teamname
            teamname = j
            if i <= 2012 and j == 'BRK':
                j = 'NJN'
            elif i <= 2014 and j == 'CHO':
                j = 'CHA'
            elif i == 2007 and j == 'NOP':
                j = 'NOK'
            elif i >= 2008 and i <= 2013 and j == 'NOP':
                j = 'NOH'
            elif i <= 2008 and j == 'OKC':
                j = 'SEA'
                
            
            path = path1 + '\\' + teamname
            if not os.path.isdir(path):
                os.mkdir(path)

tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 20 mins

Cost 0.196496 s


# test

In [ ]:
erroryear = []
errorteam = []
errorname = []

tStart = time.time()
for i in year:
    for t in range(len(nbateam)):
        if t == 0:
            s = 'East'
        elif t == 1:
            s = 'West'
        print(str(i) + '-' + s + ' is parsing...')
        for j in nbateam[t]:
            dfplayer = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerlist\\' + str(i) + '\\' + s + '\\' + j + '\\' + j + '.xlsx')
            plist = list(dfplayer.webname)
            for k in plist:
                for a in k:
                    letter = a
                    break
                try:
                    url = 'https://www.basketball-reference.com/players/' + letter + '/' + k + '/gamelog/' + str(i) + '/'
                    tables = pd.read_html(url)
                except:
                    erroryear.append(str(i))
                    errorteam.append(j)
                    errorname.append(k)
                    continue
                try:
                    for n in tables:
                        if len(n) >= 86:
                            table = n
                    gcol = []
                    count = 0
                    for c in table['G']:
                        if c == 'G':
                            gcol.append(count)
                        count = count + 1
                    ctable = table.drop(gcol)
                    ctable = ctable.reset_index()
                    ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\' + str(i) + '\\' + s + '\\' + j + '\\' + k + '.xlsx')
                except:
                    continue
errordic = {'year' : erroryear, 'team' : errorteam, 'name' : errorname}
error = pd.DataFrame(errordic)
error.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\error.xlsx')
tEnd = time.time()
print('Cost %f s' %(tEnd - tStart))
#almost 7 hours

In [12]:
dfplayerlist = pd.read_excel('C:\\Users\\ecom\\Desktop\\NBAData\\playerList.xlsx')
url = 'https://www.basketball-reference.com/players/b/brownsh01/gamelog/2007/'
tables = pd.read_html(url)
for j in range(len(dfplayerlist.website)):
    if dfplayerlist.website[j] == url:
        pyear = dfplayerlist.year[j]
        pside = dfplayerlist.side[j]
        pteam = dfplayerlist.team[j]
        pname = dfplayerlist.name[j]
lntable = 0
for ts in tables:
    if len(ts) > lntable:
        lntable = len(ts)
        table = ts
gcol = []
count = 0
for c in table['G']:
    if c == 'G':
        gcol.append(count)
    count = count + 1
ctable = table.drop(gcol)
ctable = ctable.reset_index()
ctable.to_excel('C:\\Users\\ecom\\Desktop\\NBAData\\' + str(pyear) + '\\' + pside + '\\' + pteam + '\\' + pname + '.xlsx')

C:\Users\ecom\Desktop\NBAData\2007\East\CLE\Shannon Brown.xlsx
